# How to remove large intermediate files without breaking signatures

* **Difficulty level**: easy
* **Time need to lean**: 10 minutes or less
* **Key points**:
  * `zapped` files have only their signatures
  

### Removal of large intermediate files

SoS keep tracks of all intermediate files and will rerun steps only if any of the tracked files are removed or changed. However, it is often desired to remove some of the large non-essential intemediate files to reduce diskspace used by completed workflows, while allowing the workflow to be re-executed without these files. SoS provides a command

```
sos remove files --zap
```

to zap specified file, or for example

```
sos remove . --size +5G --zap
```
to zap all files larger than 5G. This command removes specified files but keeps a special `{file}.zapped` file with essential information (e.g. md5 signature, and size). SoS would consider a file exist when a `.zapped` file is present and will only regenerate the file if the actual file is needed for a later step.

For example, let us execute a workflow with output `temp/result.txt`, and `temp/size.txt`.

In [1]:
%run -s force
[10]
output:  "temp/result.txt"
# added comment
sh: expand=True
    dd if=/dev/urandom of={_output} count=2000

[20]
output:  'temp/size.txt'
with open(_output[0], 'w') as sz:
    sz.write(f"Modified {_input}: {os.path.getsize(_input[0])}\n")

2000+0 records in


2000+0 records out


1024000 bytes transferred in 0.045143 secs (22683528 bytes/sec)


ERROR: [20]: [0]: 


---------------------------------------------------------------------------


NameError                                 Traceback (most recent call last)


script_1891641418037190126 in <module>


      with open(_output[0], 'w') as sz:


---->     sz.write(f"Modified {_input}: {os.path.getsize(_input[0])}\n")


NameError: name 'os' is not defined


and let us zap the intermediate file `temp/result.txt`,

In [2]:
!sos remove temp/result.txt --zap -y
!ls temp

INFO: 404 tracked files are identified.


Zap tracked file temp/result.txt


INFO: 1 file zapped


result.txt.zapped


As you can see, `temp/result.txt` is replaced with `temp/result.txt.zapped`. Now if you rerun the workflow

In [3]:
%rerun -s default -v2

File contains parsing errors: 
	[line  2]: %rerun -s default -v2

Invalid statements: SyntaxError('invalid syntax', ('<string>', 1, 1, '%rerun -s default -v2\n'))


## Further reading

* 